In [9]:
# Install and Import Library
import pandas as pd
import numpy as np
import panel as pn
import hvplot.pandas

pn.extension('tabulator')

In [10]:
df = pd.read_csv('auto-mpg.csv')

In [11]:
df

,mpg,cylinders,displacement,horsepower,weight,acceleration,model year,origin,car name
0,18.0,8,307.0,130,3504,12.0,70,1,chevrolet chevelle malibu
1,15.0,8,350.0,165,3693,11.5,70,1,buick skylark 320
2,18.0,8,318.0,150,3436,11.0,70,1,plymouth satellite
3,16.0,8,304.0,150,3433,12.0,70,1,amc rebel sst
4,17.0,8,302.0,140,3449,10.5,70,1,ford torino
...,...,...,...,...,...,...,...,...,...
393,27.0,4,140.0,86,2790,15.6,82,1,ford mustang gl
394,44.0,4,97.0,52,2130,24.6,82,2,vw pickup
395,32.0,4,135.0,84,2295,11.6,82,1,dodge rampage
396,28.0,4,120.0,79,2625,18.6,82,1,ford ranger


In [12]:
df.columns

Index(['mpg', 'cylinders', 'displacement', 'horsepower', 'weight',
       'acceleration', 'model year', 'origin', 'car name'],
      dtype='object')

### Widget

In [16]:
cylinder_options = ['All'] + sorted(df['cylinders'].unique().tolist())
cylinder_selector = pn.widgets.MultiSelect(
    name='Jumlah Silinder',
    options=cylinder_options,
    value=['All'],
    size=4,
    styles={'background': 'white', 'font-family': 'Inter, sans-serif'})

year_options = ['All'] + sorted(df['model year'].unique().tolist())
year_select = pn.widgets.Select(
    name='Tahun Model',
    options=year_options,
    value='All',
    styles={'background': 'white', 'font-family': 'Inter, sans-serif'}
)

In [17]:
def filter_data(cylinders, year):
    data = df.copy()
    if 'All' not in cylinders:
        data = data[data['cylinders'].isin(cylinders)]
    if year != 'All':
        data = data[data['model year'] == int(year)]
    return data

### Pembuatan Dashboard

In [18]:
import panel as pn
import hvplot.pandas

pn.extension('tabulator')
pn.extension(design='material')

pn.config.sizing_mode = 'stretch_width'

# Card 
@pn.depends(cylinder_selector, year_select)
def summary_stats(cylinders, year):
    filtered = filter_data(cylinders, year)
    
    total_cars = len(filtered)
    avg_mpg = filtered['mpg'].mean()
    max_mpg = filtered['mpg'].max()
    min_mpg = filtered['mpg'].min()
    
    stats_html = f"""
    <link href="https://fonts.googleapis.com/css2?family=Inter:wght@400;600;700&display=swap" rel="stylesheet">
    <div style="display: grid; grid-template-columns: repeat(4, 1fr); gap: 20px; margin-bottom: 20px; font-family: 'Inter', sans-serif;">
        <div style="background: #A8DADC; padding: 25px; border-radius: 12px; box-shadow: 0 2px 8px rgba(0,0,0,0.08);">
            <div style="font-size: 13px; font-weight: 600; opacity: 0.7; text-transform: uppercase; letter-spacing: 0.5px; color: #1a1a1a;">Total Mobil</div>
            <div style="font-size: 36px; font-weight: 700; margin-top: 12px; color: #1a1a1a;">{total_cars}</div>
        </div>
        <div style="background: #F4A6A0; padding: 25px; border-radius: 12px; box-shadow: 0 2px 8px rgba(0,0,0,0.08);">
            <div style="font-size: 13px; font-weight: 600; opacity: 0.7; text-transform: uppercase; letter-spacing: 0.5px; color: #1a1a1a;">Rata-rata MPG</div>
            <div style="font-size: 36px; font-weight: 700; margin-top: 12px; color: #1a1a1a;">{avg_mpg:.1f}</div>
        </div>
        <div style="background: #B8D4E8; padding: 25px; border-radius: 12px; box-shadow: 0 2px 8px rgba(0,0,0,0.08);">
            <div style="font-size: 13px; font-weight: 600; opacity: 0.7; text-transform: uppercase; letter-spacing: 0.5px; color: #1a1a1a;">MPG Tertinggi</div>
            <div style="font-size: 36px; font-weight: 700; margin-top: 12px; color: #1a1a1a;">{max_mpg:.1f}</div>
        </div>
        <div style="background: #C8E6C9; padding: 25px; border-radius: 12px; box-shadow: 0 2px 8px rgba(0,0,0,0.08);">
            <div style="font-size: 13px; font-weight: 600; opacity: 0.7; text-transform: uppercase; letter-spacing: 0.5px; color: #1a1a1a;">MPG Terendah</div>
            <div style="font-size: 36px; font-weight: 700; margin-top: 12px; color: #1a1a1a;">{min_mpg:.1f}</div>
        </div>
    </div>
    """
    return pn.pane.HTML(stats_html)

# Grafik distribusi MPG 
@pn.depends(cylinder_selector, year_select)
def mpg_distribution(cylinders, year):
    filtered = filter_data(cylinders, year)
    return filtered.hvplot.hist(
        'mpg', 
        bins=15, 
        color='#A8DADC',
        alpha=0.8,
        xlabel='Miles per Gallon',
        ylabel='Frekuensi',
        height=350,
        responsive=True
    )
    
# Grafik rata-rata MPG per Silinder 
@pn.depends(cylinder_selector, year_select)
def avg_mpg_per_cyl(cylinders, year):
    filtered = filter_data(cylinders, year)
    avg = filtered.groupby('cylinders')['mpg'].mean().reset_index()
    return avg.hvplot.bar(
        x='cylinders', 
        y='mpg', 
        color='#F4A6A0',
        alpha=0.8,
        xlabel='Jumlah Silinder',
        ylabel='MPG',
        height=350,
        responsive=True
    )

# Grafik hubungan Weight vs MPG
@pn.depends(cylinder_selector, year_select)
def weight_vs_mpg(cylinders, year):
    filtered = filter_data(cylinders, year)
    return filtered.hvplot.scatter(
        x='weight', 
        y='mpg', 
        by='cylinders',
        hover_cols=['car name'],
        size=80,
        alpha=0.7,
        xlabel='Berat (lbs)',
        ylabel='MPG',
        height=350,
        responsive=True,
        legend='top_right',
        cmap='Pastel1'
    )

# Layout Dashboard
dashboard = pn.template.MaterialTemplate(
    title='Dashboard Analisis Mobil',
    sidebar=[
        pn.pane.Markdown('## Filter Data', styles={'color': '#1a1a1a', 'font-family': 'Inter, sans-serif'}),
        cylinder_selector,
        year_select,
        pn.pane.Markdown('---'),
        pn.pane.Markdown(
            '**Petunjuk:**\n\n'
            '- Pilih silinder untuk filter data\n'
            '- Pilih "All" untuk semua data\n'
            '- Tahun model dapat difilter dari dropdown',
            styles={'font-size': '12px', 'color': '#1a1a1a', 'font-family': 'Inter, sans-serif', 'opacity': '0.8'}
        )
    ],
    main=[
        pn.Row(summary_stats),
        pn.Row(
            pn.Card(
                mpg_distribution, 
                title='Distribusi MPG',
                collapsible=False,
                styles={'box-shadow': '0 2px 8px rgba(0,0,0,0.08)', 'color': '#1a1a1a'},
                header_color='#1a1a1a'
            ),
            pn.Card(
                avg_mpg_per_cyl, 
                title='Rata-rata MPG per Silinder',
                collapsible=False,
                styles={'box-shadow': '0 2px 8px rgba(0,0,0,0.08)', 'color': '#1a1a1a'},
                header_color='#1a1a1a'
            )
        ),
        pn.Row(
            pn.Card(
                weight_vs_mpg, 
                title='Hubungan Berat vs MPG',
                collapsible=False,
                styles={'box-shadow': '0 2px 8px rgba(0,0,0,0.08)', 'color': '#1a1a1a'},
                header_color='#1a1a1a'
            )
        )
    ],
    header_background='#457B9D'
)

dashboard.servable()

MaterialTemplate
    [js_area] HTML(None, design=<class 'panel.theme.materi..., height=0, margin=0, sizing_mode='fixed', width=0)
    [actions] MaterialTemplateActions()
    [browser_info] BrowserInfo(dark_mode=True, device_pixel_ratio=1.2000000476837158, language='en-US', timezone='Asia/Jakarta', timezone_offset=-420, webdriver=False, webgl=True)
    [busy_indicator] LoadingSpinner(height=20, width=20)
    [main-2453379433424] Row(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [0] ParamFunction(function, _pane=HTML, defer_load=False, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [main-2453380332496] Row(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [0] Card(collapsible=False, design=<class 'panel.theme.materi..., header_color='#1a1a1a', sizing_mode='stretch_width', styles={'box-shadow': '0 2px 8px ...}, title='Distribusi MPG')
            [0] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [1] Card(collapsible=False, design=<class 'panel.theme.materi..., header_color='#1a1a1a', sizing_mode='stretch_width', styles={'box-shadow': '0 2px 8px ...}, title='Rata-rata MPG p...)
            [0] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [main-2453380827632] Row(design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
        [0] Card(collapsible=False, design=<class 'panel.theme.materi..., header_color='#1a1a1a', sizing_mode='stretch_width', styles={'box-shadow': '0 2px 8px ...}, title='Hubungan Berat vs MPG')
            [0] ParamFunction(function, _pane=HoloViews, defer_load=False, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [nav-2453378068512] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width', styles={'color': '#1a1a1a', ...})
    [nav-2453378111504] MultiSelect(design=<class 'panel.theme.materi..., name='Jumlah Silinder', options=['All', 3, 4, 5, 6, 8], sizing_mode='stretch_width', styles={'background': 'white', ...}, value=['All'])
    [nav-2453378111184] Select(design=<class 'panel.theme.materi..., name='Tahun Model', options=['All', 70, 71, ...], sizing_mode='stretch_width', styles={'background': 'white', ...}, value='All')
    [nav-2453378068784] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width')
    [nav-2453377916752] Markdown(str, design=<class 'panel.theme.materi..., sizing_mode='stretch_width', styles={'font-size': '12px', ...})

### Insight